<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/faiss_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.pinecone.io/learn/series/faiss/faiss-tutorial/
## What is Faiss?
在我们开始编写任何代码之前，很多人都会问——Faiss 是什么？

Faiss 是一个由 Facebook AI 开发的库，可实现高效的相似性搜索。

因此，给定一组向量，我们可以使用 Faiss 对它们进行索引 - 然后使用另一个向量（查询向量），我们在索引中搜索最相似的向量。

现在，Faiss 不仅允许我们构建索引和搜索，而且还可以将搜索时间加快到更好的性能水平，这是我们将在本文中探讨的内容。





In [ ]:
!apt install libomp-dev
!pip install --upgrade faiss-cpu faiss-gpu

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libomp-14-dev libomp5-14
Suggested packages:
  libomp-14-doc
The following NEW packages will be installed:
  libomp-14-dev libomp-dev libomp5-14
0 upgraded, 3 newly installed, 0 to remove and 18 not upgraded.
Need to get 738 kB of archives.
After this operation, 8,991 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp5-14 amd64 1:14.0.0-1ubuntu1.1 [389 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libomp-14-dev amd64 1:14.0.0-1ubuntu1.1 [347 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libomp-dev amd64 1:14.0-55~exp2 [3,074 B]
Fetched 738 kB in 1s (700 kB/s)
Selecting previously unselected package libomp5-14:amd64.
(Reading database ... 120895 files and directories currently installed.)
Preparing to unpack .../libomp5-14_1%3a14

In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=8774ca22534c243bde43895901cae8c9cdcfe1945a6d9c5df5d008738bde2bce
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


## Building Some Vectors
我们需要的第一件事是数据，我们将从这个语义测试相似性中心存储库中连接多个数据集。我们将下载每个数据集，并将相关文本列提取到单个列表中。




In [ ]:
import requests
from io import StringIO
import pandas as pd
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')
# create dataframe
data = pd.read_csv(StringIO(res.text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


In [ ]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

In [ ]:
# we take all samples from both sentence A and B
sentences = data['sentence_A'].tolist()
sentence_b = data['sentence_B'].tolist()
sentences.extend(sentence_b)  # merge them
len(set(sentences))  # together we have ~4.5K unique sentences

4802

In [ ]:
urls = [
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv'
]
# each of these dataset have the same structure, so we loop through each creating our sentences data
for url in urls:
    res = requests.get(url)
    # extract to dataframe
    dataframe = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
    # add to columns 1 and 2 to sentences list
    sentences.extend(dataframe[1].tolist())
    sentences.extend(dataframe[2].tolist())
len(set(sentences))

<ipython-input-6-c448ed2f868a>:14: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  dataframe = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
Skipping line 191: expected 3 fields, saw 4
Skipping line 206: expected 3 fields, saw 4
Skipping line 295: expected 3 fields, saw 4
Skipping line 695: expected 3 fields, saw 4
Skipping line 699: expected 3 fields, saw 4

<ipython-input-6-c448ed2f868a>:14: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  dataframe = pd.read_csv(StringIO(res.text), sep='\t', header=None, error_bad_lines=False)
Skipping line 104: expected 3 fields, saw 4
Skipping line 181: expected 3 fields, saw 4
Skipping line 317: expected 3 fields, saw 4
Skipping line 412: expected 3 fields, saw 5
Skipping line 508: expected 3 fields, saw 4

<ipython-input-6-c448ed2f8

14505

In [ ]:
# Next, we remove any duplicates, leaving us with 14.5K unique sentences.
# Finally, we build our dense vector representations of each sentence using the sentence-BERT library.

# remove duplicates and NaN
sentences = [word for word in list(set(sentences)) if type(word) is str]

from sentence_transformers import SentenceTransformer
# initialize sentence transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens')
# create sentence embeddings
sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape


(14504, 768)

一旦我们安装了 Faiss，我们就可以打开 Python 并使用IndexFlatL2构建我们的第一个、简单明了的索引。



## IndexFlatL2
IndexFlatL2测量查询向量与加载到索引中的向量之间的所有给定点之间的 L2（或欧几里德）距离。它很简单，非常准确，但不太快。


![](https://cdn.sanity.io/images/vr8gru94/production/ea951a4be3acf9d379cc6f922be1468b37b7f9e5-1280x720.png)
L2 distance calculation between a query vector xq and our indexed vectors (shown as y)



在 Python 中，我们将使用向量维度（768 - 句子嵌入的输出大小）初始化IndexFlatL2索引，如下所示：



In [ ]:
import faiss
d = sentence_embeddings.shape[1]
d

768

In [ ]:
index = faiss.IndexFlatL2(d)
index.is_trained

True

通常，我们会使用需要在加载数据之前对其进行训练的索引。我们可以使用is_trained方法检查索引是否需要训练。IndexFlatL2不是需要训练的索引，因此我们应该返回False。

准备好后，我们加载嵌入并进行查询，如下所示：




In [ ]:
index.add(sentence_embeddings)
index.ntotal


14504

In [ ]:
# Then search given a query `xq` and number of nearest neigbors to return `k`.
k = 4
xq = model.encode(["Someone sprints with a football"])

In [ ]:
%%time
D, I = index.search(xq, k)  # search
# Here we're returning indices
print(I,D)

[[6880 3097 2274 1437]] [[54.623795 54.8535   57.356262 57.90586 ]]
CPU times: user 14.6 ms, sys: 5 µs, total: 14.6 ms
Wall time: 16.4 ms


现在，如果我们想从 Faiss 中提取数值向量，我们也可以这样做。



In [ ]:
!pip install numpy

In [ ]:
import numpy as np

# we have 4 vectors to return (k) - so we initialize a zero array to hold them
vecs = np.zeros((k, d))
# then iterate through each ID from I and add the reconstructed vector to our zero-array
for i, val in enumerate(I[0].tolist()):
    vecs[i, :] = index.reconstruct(val)

In [ ]:
vecs.shape

(4, 768)

In [ ]:
vecs[0][:100]


array([ 0.01627017,  0.22325909, -0.15037386, -0.30747277, -0.27122417,
       -0.10593203, -0.06460906,  0.04738198, -0.73349065, -0.37657702,
       -0.76762819,  0.1690291 ,  0.53107673,  0.51176614,  1.14415824,
       -0.08562881, -0.67240089, -0.96637088,  0.02545483, -0.21559834,
       -1.25656641, -0.82982188, -0.09824977, -0.21850878,  0.5061025 ,
        0.10527938,  0.50396883,  0.6524297 , -1.39458668,  0.65847492,
       -0.21525349, -0.22487436,  0.81818324,  0.08464295, -0.76141709,
       -0.2892831 , -0.09825782, -0.7304613 ,  0.07855817, -0.84354568,
       -0.59242076,  0.7747137 , -1.20920575, -0.2275797 , -1.30733621,
       -0.23081495, -1.31322515,  0.01629095, -0.97285444,  0.19308169,
        0.47424543,  1.18920887, -1.96741247, -0.70061117, -0.29638764,
        0.60533708,  0.62407458, -0.70340389, -0.86754179,  0.17673109,
       -0.19170582, -0.02951964,  0.2262352 , -0.16695435, -0.80402535,
       -0.45918941,  0.69675511, -0.24928196, -1.01478696, -0.92

## Speed
单独使用IndexFlatL2索引的计算成本很高，并且扩展性不佳。

使用此索引时，我们正在执行详尽的搜索 - 这意味着我们将查询向量xq与索引中的每个其他向量进行比较，在我们的例子中，每次搜索的 L2 距离计算为 14.5K。

想象一下我们搜索包含 1M、1B 甚至更多向量的数据集的速度 - 当我们包含多个查询向量时？


![](https://cdn.sanity.io/images/vr8gru94/production/2a7cf4de5beb7a8addb82e6f899b24dd455847fa-1280x720.png)
Milliseconds taken to return a result (y-axis) / number of vectors in the index (x-axis) — relying solely on IndexFlatL2 quickly becomes slow

我们的索引很快就会变得太慢而无法使用，因此我们需要做一些不同的事情。





# Partitioning The Index
Faiss 允许我们添加多个步骤，使用许多不同的方法来优化我们的搜索。一种流行的方法是将索引划分为 Voronoi 单元。


![](https://cdn.sanity.io/images/vr8gru94/production/ca1ed9b80fd0788cee513ef75c1b8bd8daad8571-1400x748.png)
我们可以想象我们的向量都包含在一个 Voronoi 单元中——当我们引入一个新的查询向量时，我们首先测量它的质心之间的距离，然后将我们的搜索范围限制在该质心的单元内。


使用此方法，我们将获取查询向量xq，识别它所属的单元格，然后使用我们的IndexFlatL2（或其他度量）在查询向量和属于该特定单元格的所有其他向量之间进行搜索。

因此，我们正在缩小搜索范围，产生近似答案，而不是精确答案（通过穷举搜索产生）。

为了实现这一点，我们首先使用IndexFlatL2初始化索引- 但这一次，我们使用 L2 索引作为量化器步骤 - 我们将其输入到分区IndexIVFFlat索引中。





In [ ]:
nlist = 50  # how many cells
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

这里我们添加了一个新参数nlist。我们使用nlist来指定我们希望索引有多少个分区（Voronoi 单元）。

现在，当我们构建之前的IndexFlatL2 -only 索引时，我们不需要训练索引，因为构建索引不需要分组/转换。因为我们使用IndexIVFFlat添加了集群，所以情况不再如此。

因此，我们现在要做的就是根据数据训练索引——在将任何数据添加到索引之前我们必须这样做。





In [ ]:
index.is_trained

False

In [ ]:
index.train(sentence_embeddings)
index.is_trained  # check if index is now trained

True

In [ ]:
index.add(sentence_embeddings)
index.ntotal  # number of embeddings indexed

14504

现在我们的索引已经训练完毕，我们像以前一样添加数据。

让我们使用相同的索引句子嵌入和相同的查询向量xq再次搜索。



In [ ]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[6880 3097 2274 1437]]
CPU times: user 1.7 ms, sys: 6 µs, total: 1.71 ms
Wall time: 1.86 ms


搜索时间明显减少，在这种情况下，我们没有发现穷举搜索和近似搜索返回的结果之间有任何差异。但是，情况常常如此。

如果使用IndexIVFFlat进行近似搜索返回次优结果，我们可以通过增加搜索范围来提高准确性。我们通过增加nprobe属性值来实现这一点——它定义了要搜索的附近单元格数量。


![](https://cdn.sanity.io/images/vr8gru94/production/f32a71b57eefa87ef461bb3412f9fc21bbd46514-2020x1270.png)
当 nprobe == 1 时搜索单个最近的单元格（左），当 nprobe == 8 时搜索八个最近的单元格（右）


我们可以很容易地实现这个改变。




In [ ]:
# We can increase the number of nearby cells to search too with `nprobe`.
index.nprobe = 10


In [ ]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[6880 3097 2274 1437]]
CPU times: user 2.47 ms, sys: 0 ns, total: 2.47 ms
Wall time: 3.31 ms


现在，因为我们通过增加nprobe值来搜索更大的范围，所以我们也会看到搜索速度增加。


![](https://cdn.sanity.io/images/vr8gru94/production/84b1a10186cdd9dec8ebcfff9a96dbc89951d4b6-1280x720.png)
Query time / number of vectors for the IVFFlat index with different nprobe values — 1, 5, 10, and 20


尽管如此，即使使用较大的nprobe值，我们仍然看到比仅使用IndexFlatL2索引返回的响应快得多。





# Vector Reconstruction
如果我们继续尝试再次使用`index.reconstruct(<vector_idx>)`，我们将返回RuntimeError，因为由于添加了 IVF 步骤，原始向量与其索引位置之间没有直接映射。

因此，如果我们想重建向量，我们必须首先使用`index.make_direct_map()`创建这些直接映射。




In [ ]:
index.make_direct_map()
index.reconstruct(7460)[:100]



array([-0.21456271,  0.8143777 ,  1.0099413 ,  0.09222746, -0.16850969,
       -0.7659664 ,  0.4287243 ,  0.75882006,  0.5985496 ,  0.0871104 ,
       -0.4260962 ,  0.6333268 ,  0.5582458 , -0.5326644 ,  0.61365986,
        0.46745   , -0.42246723, -0.9023604 ,  0.4810551 , -0.40765068,
       -0.70955026, -0.2697846 , -0.69924927, -0.9644324 , -0.44599137,
       -0.65064913,  0.09697644, -1.2216316 , -1.1324755 ,  0.7620295 ,
        0.39830536,  0.95234334,  0.2690042 , -0.4944913 ,  1.0677664 ,
        1.444611  ,  1.0232674 ,  0.11022919, -0.43518692,  0.16352642,
        1.5095773 ,  0.3366695 ,  0.47655424,  0.01071994, -0.416367  ,
       -0.5215626 ,  0.7465224 ,  0.26187274,  0.41421804, -0.62833065,
        0.25291294, -0.91737694, -0.13963881, -0.23961976, -0.47167268,
        0.5198275 ,  0.303813  , -1.1317794 , -0.12226551, -0.5565425 ,
        0.23279233,  0.37011403,  0.4685905 , -0.03408442, -0.03304414,
        0.62023747,  0.8740958 ,  0.6763916 , -0.11267618, -0.63

## Quantization
我们还有一项关键优化需要介绍。到目前为止，我们所有的索引都将向量存储为完整（例如Flat）向量。现在，在非常大的数据集中，这很快就会成为一个问题。

幸运的是，Faiss 能够使用乘积量化 (PQ)来压缩我们的向量。

但是，什么是PQ？好吧，我们可以将其视为一个额外的近似步骤，其结果与我们使用IVF类似。IVF 允许我们通过缩小搜索范围来进行近似，而 PQ 则通过距离/相似度计算进行近似。

PQ 通过压缩向量本身来实现这种近似相似性运算，该操作由三个步骤组成。

![](https://cdn.sanity.io/images/vr8gru94/production/6eb8071e80abf8fa8d6c170270efd5db52a3168f-1400x787.png)

乘积量化三步走


1. 我们将原始向量分成几个子向量。

2. 对于每组子向量，我们执行聚类操作 - 为每个子向量集创建多个质心。

3. 在我们的子向量向量中，我们将每个子向量替换为其最近的特定于集合的质心的 ID。


为了实现这一切，我们使用 IndexIVF PQ索引——我们还需要在添加嵌入之前**训练**索引。



In [ ]:
m = 8  # number of centroid IDs in final compressed vectors
bits = 8 # number of bits in each centroid

quantizer = faiss.IndexFlatL2(d)  # we keep the same L2 distance flat index
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, bits)

In [ ]:
#`train` the index.
print(index.is_trained)
index.train(sentence_embeddings)
print(index.is_trained)
#And `add` our vectors.
index.add(sentence_embeddings)

True
True


现在我们准备开始使用新索引进行搜索。



In [ ]:
index.nprobe = 10  # align to previous IndexIVFFlat nprobe value

In [ ]:
%%time
D, I = index.search(xq, k)
print(I)

[[ 337  982 1437 1660]]
CPU times: user 1.23 ms, sys: 994 µs, total: 2.22 ms
Wall time: 1.97 ms


## Speed or Accuracy?
通过添加 PQ，我们将 IVF 搜索时间从约 7.5 毫秒减少到约 5 毫秒，对于这种大小的数据集来说，这是一个很小的差异 - 但当扩大规模时，这一点很快就会变得显着。

但是，我们还应该注意返回的结果略有不同。之前，通过详尽的 L2 搜索，我们返回了[[6880 3097 2274 1437]]。而PQ量化之后的结果：[[ 337  982 1437 1660]]。

我们的两种速度优化操作IVF和PQ都是以准确性为代价的。现在，如果我们打印出这些结果，我们仍然会发现每一项都是相关的：




In [ ]:
[f'{i}: {sentences[i]}' for i in I[0]]



['337: A football player is running past an official carrying a football',
 '982: Football players are on the field.',
 '1437: A person playing football is running past an official carrying a football',
 '1660: A football player is running into an official carrying a football']

因此，尽管我们可能无法得到完美的结果，但我们仍然很接近——并且由于近似值，我们得到了更快的响应。

![](https://cdn.sanity.io/images/vr8gru94/production/f0a368ac2ff6372615fef4eb3c30e89bfd54c22d-1280x720.png)
三个索引的查询时间/向量数量


而且，如上图所示，随着索引大小的增加，查询时间的差异变得越来越重要。

这就是本文的内容！我们已经介绍了开始在 Faiss 中构建高性能搜索索引的要点。

显然，使用IndexFlatL2、IndexIVFFlat和IndexIVFPQ可以做很多事情，并且每个参数都有许多参数，可以根据我们特定的精度/速度要求进行微调。如图所示，借助 Faiss，我们可以轻松地以闪电般的速度产生一些真正令人印象深刻的结果。

